### *Setup*

In [ ]:
!pip install torch
!pip install datasets
!pip install librosa
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
# import libraries
import datasets
from datasets import load_dataset, DatasetDict,  Audio
import pandas as pd
import os
import glob
import librosa
import io
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score
from transformers import WhisperModel, WhisperFeatureExtractor, AdamW
import torch
import torch.nn as nn
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from sklearn.metrics import f1_score, classification_report, accuracy_score

In [ ]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
data_path = "/content/drive/MyDrive/DSI/data/WAV_PUBLIC_SAMPLES"

train_catalog = "/content/drive/MyDrive/DSI/data/train.csv"
test_catalog = "/content/drive/MyDrive/DSI/data/test.csv"

In [ ]:
train_metadata = pd.read_csv(train_catalog)

In [ ]:
# add cols for wav data

# Replace ".mp3" with ".wav" in the "Filename" column
train_metadata['WAV_filename'] = train_metadata['File_Name'].str.replace('.mp3', '.wav')

# Create "WAV_folder" column by concatenating "_WAV" to the "folder" column
train_metadata['WAV_folder'] = train_metadata['folder'] + "_WAV"

train_metadata

<ipython-input-8-54011f6dbdc7>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  train_metadata['WAV_filename'] = train_metadata['File_Name'].str.replace('.mp3', '.wav')


,File_Name,Sampling_Rate_(Hz),Channels,Duration_(seconds),folder,hypernasality,original_text,OPENAI_Whisper_text,WAV_filename,WAV_folder
0,ACPA ted had a dog with white feet-3.mp3,44100.0,1.0,4.13,CASES,1.0,ted had a dog with white feet,Ted and a dog with white feet.,ACPA ted had a dog with white feet-3.wav,CASES_WAV
1,cdc 4 (and then go to school).mp3,44100.0,2.0,1.41,CONTROLS,0.0,and then go to school,and then go to school.,cdc 4 (and then go to school).wav,CONTROLS_WAV
2,Video 1_4 (and can I have some more material).mp3,44100.0,2.0,3.60,CONTROLS,0.0,and can I have some more material,And can I have some more material?,Video 1_4 (and can I have some more material).wav,CONTROLS_WAV
3,NEW - video 2 (three times).mp3,44100.0,2.0,1.28,CONTROLS,0.0,three times,Three times.,NEW - video 2 (three times).wav,CONTROLS_WAV
4,cdc 4 (and then he brushed his teeth).mp3,44100.0,2.0,1.52,CONTROLS,0.0,and then he brushed his teeth,And then he brushed his teeth.,cdc 4 (and then he brushed his teeth).wav,CONTROLS_WAV
...,...,...,...,...,...,...,...,...,...,...
142,video 1 (pizza bundt).mp3,44100.0,2.0,1.80,CONTROLS,0.0,pizza bundt,Pizza Funt!,video 1 (pizza bundt).wav,CONTROLS_WAV
143,ACPA most boys like to play football-3.mp3,48000.0,1.0,3.31,CASES,1.0,most boys like to play football,Most boys like to play football.,ACPA most boys like to play football-3.wav,CASES_WAV
144,Facebook (take a tire).mp3,44100.0,1.0,1.75,CASES,1.0,take a tire,See you next time!,Facebook (take a tire).wav,CASES_WAV
145,Video 5_1 (feet).mp3,44100.0,2.0,1.04,CASES,1.0,feet,Peace.,Video 5_1 (feet).wav,CASES_WAV


In [ ]:
train_df, val_df = train_test_split(train_metadata, test_size = 0.3, random_state = 42)

In [ ]:
train_files = train_df["WAV_filename"].tolist()

train_folder = train_df["WAV_folder"].tolist()

train_full_paths = [os.path.join(data_path,train_folder[i], train_files[i]) for i in range(0,len(train_files))]

In [ ]:
train_labels = train_df["hypernasality"].tolist()

train_labels[0:10]

[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]

In [ ]:
# val set
val_files = val_df["WAV_filename"].tolist()

val_folder = val_df["WAV_folder"].tolist()

val_full_paths = [os.path.join(data_path,val_folder[i], val_files[i]) for i in range(0,len(val_files))]

val_labels = val_df["hypernasality"].tolist()

# Feature Extractor

###Create PyTorch Dataset

In [ ]:
# train dataset only

train_audio_dataset = datasets.Dataset.from_dict({"audio": train_full_paths,
                                                  "labels":train_labels}
                                                 ).cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
model_checkpoint = "openai/whisper-base"

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_checkpoint)
encoder = WhisperModel.from_pretrained(model_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

In [ ]:
class SpeechClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, audio_data,  text_processor):
        self.audio_data = audio_data
        self.text_processor = text_processor

    def __len__(self):
        return len(self.audio_data)

    def __getitem__(self, index):

      inputs = self.text_processor(self.audio_data[index]["audio"]["array"],
                                   return_tensors="pt",
                                   sampling_rate=self.audio_data[index]["audio"]["sampling_rate"])
      input_features = inputs.input_features
      decoder_input_ids = torch.tensor([[1, 1]]) * encoder.config.decoder_start_token_id

      labels = np.array(self.audio_data[index]['labels'])

      return input_features, decoder_input_ids, torch.tensor(labels)

In [ ]:
train_dataset = SpeechClassificationDataset(train_audio_dataset, feature_extractor)

batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

### Whisper Feature Extractor

In [ ]:
model_checkpoint = "openai/whisper-base"

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_checkpoint)
encoder = WhisperModel.from_pretrained(model_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
feature_extractor

WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [ ]:
import torchaudio
import matplotlib.pyplot as plt

# Function to extract spectrogram from an audio file using the feature extractor
def extract_spectrogram(train_full_paths):
    waveform, sample_rate = torchaudio.load(train_full_paths)

    # Extract features using the feature extractor
    features = feature_extractor(waveform.squeeze().numpy(), return_tensors="pt").input_values

    # You can choose to visualize the features or use them in downstream tasks
    return features

# Function to visualize spectrogram
def visualize_spectrogram(filename):
    #file_path = os.path.join(train_full_paths, filename)

    # Extract features (including spectrogram) for each audio file
    features = extract_spectrogram(filename)

    # Convert features to a NumPy array for visualization
    features_np = features.squeeze().numpy()

    # Plot and show the spectrogram (or any other features)
    plt.imshow(features_np, aspect="auto", origin="lower")
    plt.title("Spectrogram (or other features)")
    plt.xlabel("Time")
    plt.ylabel("Feature Dimension")
    plt.show()

visualize_spectrogram(train_full_paths[15])

RuntimeError: Failed to open the input "/content/drive/MyDrive/DSI/data/WAV_PUBLIC_SAMPLES/CONTROLS_WAV/cdc 4 (and then he ate breakfast).wav" (No such file or directory).
Exception raised from get_input_format_context at /__w/audio/audio/pytorch/audio/src/libtorio/ffmpeg/stream_reader/stream_reader.cpp:42 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x78d3d14b3d87 in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x78d3d146475f in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x42904 (0x78d385dca904 in /usr/local/lib/python3.10/dist-packages/torio/lib/libtorio_ffmpeg4.so)
frame #3: torio::io::StreamingMediaDecoder::StreamingMediaDecoder(std::string const&, std::optional<std::string> const&, std::optional<std::map<std::string, std::string, std::less<std::string>, std::allo

In [ ]:
train_full_paths

['/content/drive/MyDrive/DSI/data/WAV_PUBLIC_SAMPLES/CONTROLS_WAV/ACPA Santa came home since the snow fell.wav',
 '/content/drive/MyDrive/DSI/data/WAV_PUBLIC_SAMPLES/CONTROLS_WAV/cdc 5 (can I play with Jack).wav',
 '/content/drive/MyDrive/DSI/data/WAV_PUBLIC_SAMPLES/CONTROLS_WAV/cdc 6 (the polar bears are dancing).wav',
 '/content/drive/MyDrive/DSI/data/WAV_PUBLIC_SAMPLES/CASES_WAV/ACPA ted had a dog with white feet-3.wav',
 '/content/drive/MyDrive/DSI/data/WAV_PUBLIC_SAMPLES/CASES_WAV/Video 1_4 (seesaw).wav',
 '/content/drive/MyDrive/DSI/data/WAV_PUBLIC_SAMPLES/CONTROLS_WAV/ACPA Ted had a dog with white feet.wav',
 '/content/drive/MyDrive/DSI/data/WAV_PUBLIC_SAMPLES/CONTROLS_WAV/cdc 7 (yellow).wav',
 '/content/drive/MyDrive/DSI/data/WAV_PUBLIC_SAMPLES/CASES_WAV/ACPA in the safe deposit department.wav',
 '/content/drive/MyDrive/DSI/data/WAV_PUBLIC_SAMPLES/CASES_WAV/Video 6_3 (puppy will pull a rope).wav',
 '/content/drive/MyDrive/DSI/data/WAV_PUBLIC_SAMPLES/CASES_WAV/ACPA opening and c